In [2]:
import folium
from branca.element import Figure
import altair as alt

import json
import numpy as np
import vincent


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from statsmodels.api import OLS, add_constant
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose

plt.rcParams['figure.figsize'] = (12,6)


In [4]:
m = folium.Map(location=[52.518811, 13.412157],
               #tiles='Stamen Terrain',
               tiles='Stamen Toner',
               #tiles='Mapbox Control Room',
              )
m


In [5]:
stations = pd.read_csv('stations.txt', sep =',' , skiprows = 17)
stations = pd.DataFrame(stations)
stations.columns=stations.columns.map(str.strip)
stations.columns

Index(['STAID', 'STANAME', 'CN', 'LAT', 'LON', 'HGHT'], dtype='object')

In [6]:
stations

,STAID,STANAME,CN,LAT,LON,HGHT
0,257,CET CENTRAL ENGLAND,GB,+52:25:12,-001:49:48,78
1,271,ARMAGH,GB,+54:21:00,-006:39:00,62
2,272,ESKDALEMUIR,GB,+55:19:00,-003:12:00,242
3,274,RADCLIFFE METEOROLOGICAL STATION OXFORD,GB,+51:45:40,-001:15:50,63
4,273,HULL,GB,+53:45:21,-000:21:23,2
...,...,...,...,...,...,...
115,11109,PERSHORE COLLEGE,GB,+52:05:59,-002:03:31,37
116,11110,SUTTON BONINGTON,GB,+52:50:10,-001:14:59,43
117,11111,WIGHT ST CATHERINES POINT,GB,+50:34:30,-001:17:44,20
118,11112,WISLEY,GB,+51:18:36,-000:28:29,38


In [7]:
stations['STAID'] = stations['STAID'].astype(str)

In [8]:
stations['STAID_full'] = stations['STAID']
for i in stations.index:
    if len(stations.iloc[i]['STAID'])<5:
        blanks = 5-len(stations.iloc[i]['STAID'])
        stations['STAID_full'][i] = ('0'*blanks)+stations.iloc[i]['STAID']

stations['STAID'] = stations['STAID_full']
stations.drop('STAID_full', axis=1, inplace=True)
stations


<ipython-input-8-5e7066eb24a6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stations['STAID_full'][i] = ('0'*blanks)+stations.iloc[i]['STAID']


,STAID,STANAME,CN,LAT,LON,HGHT
0,00257,CET CENTRAL ENGLAND,GB,+52:25:12,-001:49:48,78
1,00271,ARMAGH,GB,+54:21:00,-006:39:00,62
2,00272,ESKDALEMUIR,GB,+55:19:00,-003:12:00,242
3,00274,RADCLIFFE METEOROLOGICAL STATION OXFORD,GB,+51:45:40,-001:15:50,63
4,00273,HULL,GB,+53:45:21,-000:21:23,2
...,...,...,...,...,...,...
115,11109,PERSHORE COLLEGE,GB,+52:05:59,-002:03:31,37
116,11110,SUTTON BONINGTON,GB,+52:50:10,-001:14:59,43
117,11111,WIGHT ST CATHERINES POINT,GB,+50:34:30,-001:17:44,20
118,11112,WISLEY,GB,+51:18:36,-000:28:29,38


In [9]:
fig=Figure()
m = folium.Map(location=[+52.2512,-00.14948],
               #tiles='Stamen Terrain',
               #tiles='Stamen Toner',
               #tiles='Mapbox Control Room',
              )
fig.add_child(m)
folium.TileLayer('Stamen Terrain').add_to(m)
folium.TileLayer('Stamen Toner').add_to(m)
folium.TileLayer('Stamen Water Color').add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)
folium.TileLayer('cartodbdark_matter').add_to(m)
folium.LayerControl().add_to(m)



In [10]:
def convert_degrees_to_latlong(coord):
    d = int(coord.split(":")[0])
    mins = int(coord.split(":")[1])
    secs = int(coord.split(":")[2])
    if d < 0:
        mins = -mins
        secs = -secs
    
    
    dd = d + (mins/60) + (secs/3600)
    return round(dd, 7)

for i,r in stations.iterrows():
    latitude = convert_degrees_to_latlong(r['LAT'])
    longitude = convert_degrees_to_latlong(r['LON'])


    #print(r['LAT'],latitude,'\n', r['LON'], longitude)
    folium.Marker(location=[latitude, longitude],
                  popup = r['STANAME'],
                  #tooltip='Click for more information!'
                 ).add_to(m)
m

In [11]:
stations['STAID'][0]

'00257'

In [12]:
staid = stations['STAID'][0]
weather_data = pd.read_csv('TG_STAID0'+staid+'.txt', sep =',' , skiprows = 19)
df= pd.DataFrame(weather_data)
df['STAID'] = staid
df

,SOUID,DATE,TG,Q_TG,STAID
0,100897,18810101,39,0,00257
1,100897,18810102,50,0,00257
2,100897,18810103,52,0,00257
3,100897,18810104,47,0,00257
4,100897,18810105,44,0,00257
...,...,...,...,...,...
50945,100897,20200626,-9999,9,00257
50946,100897,20200627,-9999,9,00257
50947,100897,20200628,-9999,9,00257
50948,100897,20200629,-9999,9,00257


In [13]:
frames = []
frames.append(df)


In [14]:
for i,r in stations.iterrows():
    if i>0:
        staid = r['STAID']
        weather_data = pd.read_csv('TG_STAID0'+staid+'.txt', sep =',' , skiprows = 19)
        df2= pd.DataFrame(weather_data)
        df2['STAID']= staid
        frames.append(df2)

In [15]:
frames

[        SOUID      DATE     TG   Q_TG  STAID
 0      100897  18810101     39      0  00257
 1      100897  18810102     50      0  00257
 2      100897  18810103     52      0  00257
 3      100897  18810104     47      0  00257
 4      100897  18810105     44      0  00257
 ...       ...       ...    ...    ...    ...
 50945  100897  20200626  -9999      9  00257
 50946  100897  20200627  -9999      9  00257
 50947  100897  20200628  -9999      9  00257
 50948  100897  20200629  -9999      9  00257
 50949  100897  20200630  -9999      9  00257
 
 [50950 rows x 5 columns],
         SOUID      DATE     TG   Q_TG  STAID
 0      146811  18440101     -3      0  00271
 1      146811  18440102    -15      0  00271
 2      146811  18440103     34      0  00271
 3      146811  18440104     80      0  00271
 4      146811  18440105     83      0  00271
 ...       ...       ...    ...    ...    ...
 64460  100927  20200626  -9999      9  00271
 64461  100927  20200627  -9999      9  00271
 6446

In [16]:
weather = pd.concat(frames)


In [17]:
weather.columns=weather.columns.map(str.strip)
weather.columns

Index(['SOUID', 'DATE', 'TG', 'Q_TG', 'STAID'], dtype='object')

In [18]:
weather['DATE']= pd.to_datetime(weather['DATE'], format='%Y%m%d')

In [19]:
weather_short = weather.query('19500101 < DATE')

In [20]:
weather_short

,SOUID,DATE,TG,Q_TG,STAID
25202,100897,1950-01-02,66,0,00257
25203,100897,1950-01-03,97,0,00257
25204,100897,1950-01-04,88,0,00257
25205,100897,1950-01-05,83,0,00257
25206,100897,1950-01-06,63,0,00257
...,...,...,...,...,...
2368,156887,2020-06-26,-9999,9,11113
2369,156887,2020-06-27,-9999,9,11113
2370,156887,2020-06-28,-9999,9,11113
2371,156887,2020-06-29,-9999,9,11113


In [21]:
weather_short.loc[weather_short['STAID']=='00257']

,SOUID,DATE,TG,Q_TG,STAID
25202,100897,1950-01-02,66,0,00257
25203,100897,1950-01-03,97,0,00257
25204,100897,1950-01-04,88,0,00257
25205,100897,1950-01-05,83,0,00257
25206,100897,1950-01-06,63,0,00257
...,...,...,...,...,...
50945,100897,2020-06-26,-9999,9,00257
50946,100897,2020-06-27,-9999,9,00257
50947,100897,2020-06-28,-9999,9,00257
50948,100897,2020-06-29,-9999,9,00257


In [22]:
data= weather_short.loc[weather_short['STAID']=='00257']

In [23]:
# Let's create the vincent chart.
scatter_points={'x':data['DATE'],
'y':data['TG']}
scatter_chart = vincent.Scatter(scatter_points,
                                iter_idx='x',
                                width=600,
                                height=300)

# Let's convert it to JSON.
scatter_json = scatter_chart.to_json()

# Let's convert it to dict.
scatter_dict = json.loads(scatter_json)



In [26]:
fig=Figure()
m2 = folium.Map(location=temp_coords,
               #tiles='Stamen Terrain',
               #tiles='Stamen Toner',
               #tiles='Mapbox Control Room',
              )
fig.add_child(m2)
folium.TileLayer('Stamen Terrain').add_to(m2)
folium.TileLayer('Stamen Toner').add_to(m2)
folium.TileLayer('Stamen Water Color').add_to(m2)
folium.TileLayer('cartodbpositron').add_to(m2)
folium.TileLayer('cartodbdark_matter').add_to(m2)
folium.LayerControl().add_to(m2)



In [25]:
temp_coords = stations.loc[stations['STAID']=='00257']
temp_coords =[convert_degrees_to_latlong(temp_coords['LAT'][0]),convert_degrees_to_latlong(temp_coords['LON'][0])]
temp_coords

[52.42, -1.83]

In [27]:
popup = folium.Popup()
folium.Vega(scatter_chart, height=350, width=650).add_to(popup)
folium.Marker(temp_coords, popup=popup).add_to(m2)

m2
#m.save(os.path.join('results', 'vega_popups.html'))

In [ ]:
# Let's create a Vega popup based on scatter_dict.
popup = folium.Popup(max_width=650)
folium.Vega(scatter_dict, height=350, width=650).add_to(popup)
folium.Marker(temp_coords, popup=popup).add_to(m2)


In [ ]:

# Let's create a Vega popup based on scatter_json.
popup = folium.Popup(max_width=0)
folium.Vega(scatter_json, height=350, width=650).add_to(popup)
folium.Marker(temp_coords, popup=popup).add_to(m2)


In [ ]:
mm2